# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Visualization of a microtubule

In [100]:
from bioexplorer import BioExplorer, Volume, Protein, AnimationParams, Vector3, Quaternion

be = BioExplorer('localhost:5000')
core = be.core_api()
# status = be.reset_scene()

### Resources

In [101]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
obj_folder = resource_folder + 'obj/'
membrane_folder = pdb_folder + 'membrane/'
lipids_folder = membrane_folder + 'lipids/'
transporters_folder = pdb_folder + 'transporters/'
tubules_folder = pdb_folder + 'tubules/'

representation = be.REPRESENTATION_ATOMS_AND_STICKS

### Membrane

In [102]:
import random

size  = Vector3(10.0, 2000.0, 0.0)
scale = 100.0

for i in range(5):
    position = Vector3(
        scale * (346 - 0) + random.randint(-scale, scale),
        scale * (1354 - 1) + random.randint(-scale, scale),
        scale * (602 - 1) + random.randint(-scale, scale))
    name = 'Microtubule%05d' % i

    # ACE2 receptor definition
    tubule = Protein(
        name=name + '_TUB',
        source=tubules_folder + '1tub.pdb', 
        rotation=Quaternion(0.0, 0.0, 0.707, 0.707),
        occurences=size.y * 2.0)

    volume = Volume(
        name=name,
        shape=be.ASSEMBLY_SHAPE_HELIX, shape_params=size,
        protein=tubule)

    status = be.add_volume(
        # atom_radius_multiplier=2.0,
        # representation=be.REPRESENTATION_DEBUG,
        position=position,
        rotation=Quaternion(0.0, 0.0, 0.0, 1.0),
        volume=volume)

    status = be.set_protein_color_scheme(
        assembly_name=name,
        name=name + '_TUB',
        color_scheme=be.COLOR_SCHEME_CHAINS,
        palette_name='Set2', palette_size=5)        

## Rendering settings

In [103]:
# status = be.reset_camera()
# status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

In [104]:
# from bioexplorer import MovieMaker
# mm = MovieMaker(be)
# mm.create_snapshot(
#     renderer='bio_explorer',
#     path='/tmp', base_name='microtubules_1k_v1',
#     size=[940, 560], samples_per_pixel=64)